In [2]:
from github import RateLimitExceededException, Github

# Providing access token
access_token = ""
g = Github(login_or_token=access_token)

# Confirm your login is successful
user = g.get_user()
print(f"Authenticated as: {user.login}")

Authenticated as: AbelvdTil


In [1]:
import os
 
STEP4_TFCOMMITS = os.path.join("data", "previous-study", "step4-tf-commits.json") 
COMMITS = os.path.join("data", "update-previous-study", "step5-commits.json")
NEW_COMMITS = os.path.join("data", "update-previous-study", "step5-new-commits.json")
STRUCTURED_NEW_COMMITS = os.path.join("data", "update-previous-study", "new-commits-dataset.json")

In [7]:
from pydriller import Repository
import json

# INITIALIZATION
terraform_output = open(STEP4_TFCOMMITS)
step4_output = json.load(terraform_output)

terraform_keywords = ['.tf', '.tf.json']
cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

iteration = 0
    
# Pull request scraping script
commit_dict = []
for rp in step4_output["repositories"]:
    try:
       

        if (iteration % 50 == 0):
            print("at iteration", iteration)
            with open(COMMITS, "w") as outfile:
                json.dump(commit_dict, outfile)

        iteration += 1

        repo = Repository(rp["name"])

        # Get each commit
        for commit in repo.traverse_commits():
            if commit.msg is not None and any(key in commit.msg.lower() for key in cost_keywords):

                # Find any commit that changes terraform file
                changed_files = []
                modified_terraform = False
                for file in commit.modified_files:
                    if any(key in file.filename for key in terraform_keywords):
                        modified_terraform = True
                        changed_files.append(file.filename)
                if modified_terraform:
                    commit_dict.append({"url": rp["name"].split(".git")[0] + "/commit/" + commit.hash, "date": str(commit.author_date), "body": commit.msg})
        
    except Exception as e:
        print("exception:", e)

with open(COMMITS, "w") as outfile:
        json.dump(commit_dict, outfile)

at iteration 0
exception: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/blinkist/terraform-aws-airship-ecs-cluster.git /tmp/tmpfqh_o9oi/terraform-aws-airship-ecs-cluster
  stderr: 'Cloning into '/tmp/tmpfqh_o9oi/terraform-aws-airship-ecs-cluster'...
fatal: could not read Username for 'https://github.com': No such device or address
'
exception: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/higef/hcloud-tf-kozak-launcher.git /tmp/tmpd13vdlvb/hcloud-tf-kozak-launcher
  stderr: 'Cloning into '/tmp/tmpd13vdlvb/hcloud-tf-kozak-launcher'...
fatal: could not read Username for 'https://github.com': No such device or address
'
exception: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/DuckbillGroup/onboarding-cft.git /tmp/tmpg_gc7bk3/onboarding-cft
  stderr: 'Cloning into '/tmp/tmpg_gc7bk3/onboarding-cft'...
fatal: could not read Username for 'https://github.com': No such devi

In [4]:
input_file = open(COMMITS)
commits = json.load(input_file)

from datetime import datetime
import pytz

utc=pytz.UTC

cutoff_time = datetime(2022,5,31,0,0,0)

with open(NEW_COMMITS, "w") as outfile:
    json.dump([commit for commit in commits if datetime.fromisoformat(commit["date"]) > utc.localize(cutoff_time)], outfile)


In [5]:
import json

input_file = open(NEW_COMMITS)
commits = json.load(input_file)

data_dict = []
for commit in commits:
    data_dict.append({"type": "commit", "url": commit["url"], "content": {"message": commit["body"]}, "codes": [""]})

with open(STRUCTURED_NEW_COMMITS, "w") as outfile:
    json.dump(data_dict, outfile)